# RAG Intro using minsearch

* [DataTalksClub: LLM RAG Workshop](https://github.com/alexeygrigorev/llm-rag-workshop/tree/main/notebooks) – parse_faq.ipynb and documents.json
* [DataTalksClub: Build Your Own Search Enginer](https://github.com/alexeygrigorev/build-your-own-search-engine) – creation of minsearch
* [minsearch repo](https://github.com/alexeygrigorev/minsearch)

In [ ]:
!pip install minsearch load_dotenv openai

  Using cached numpy-2.3.0-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scipy-1.15.3-cp313-cp313-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 26.5 MB/s eta 0:00:0000:010:01
Using cached numpy-2.3.0-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 30.0 MB/s eta 0:00:00 0:00:01
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached scipy-1.15.3-cp313-cp313-macosx_14_0_arm64.whl (22.4 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import json
import minsearch

RAG_DOCUMENT_FILENAME = 'documents.json'

def load_documents_into_minsearch():
    with open(RAG_DOCUMENT_FILENAME, 'rt') as f_in:
        docs_raw = json.load(f_in)

    documents = []
    for course_dict in docs_raw:
        for doc in course_dict['documents']:
            doc['course'] = course_dict['course']
            documents.append(doc)

    print("Document example: ", documents[0])

    index = minsearch.Index(
        text_fields=["question", "text", "section"],
        keyword_fields=["course"]
    )
    index.fit(documents)

    return index

In [4]:
RAG_DOCUMENT_COURSE_NAME = 'data-engineering-zoomcamp'

def search(index, query):
    boost = {
    'question': 3.0, # questien field is 3 times more important than text
    'section': 0.5
    }
    results = index.search(
        query=query,
        filter_dict={'course': RAG_DOCUMENT_COURSE_NAME},
        boost_dict=boost,
        num_results=2
    )
    return results

In [5]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from FAQ database.
        Use only the facts from the CONTEXT to answer the QUESTION.
        If the CONTEXT doesn't contain the answer, output NONE

        QUESTION: {question}
        CONTEXT: {context}
    """
    
    context = ""
    for doc in search_results:
        context += f'section: {doc["section"]}\nquestion: {doc["question"]}\nanswer: {doc["text"]}\n'

    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [6]:
import os
from load_dotenv import load_dotenv
load_dotenv()

assert len(os.environ['OPENAI_API_KEY']) > 1

In [7]:
from openai import OpenAI

def llm(prompt, model='gpt-4o-mini'):
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response.choices[0].message.content

In [8]:
db = load_documents_into_minsearch()

def rag(query):
    search_results = search(db, query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

Document example:  {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


In [9]:
rag('The course already started, can I still enroll?')


"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."